In [53]:
# 코드 작성일 : 24.7.4 -> 대표메뉴 추출 + 유사 메뉴 추출 
import os
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
from dateutil.relativedelta import relativedelta

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.path.dirname(original_dir))))
exec(open('setup/default.py').read())

In [2]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')

In [3]:
# 클렌징 
def filter_text(text):
    # '>'가 있는 경우 '>' 뒤의 텍스트 추출
    if '>' in text:
        text = text.replace('>,', '>') # 만약 <~>,이런식으로 붙어있으면 변경해주기
        text = text.split('>')[-1]
    # ','가 있는 경우 ',' 앞의 텍스트 추출
    if ',' in text:
        text = text.split(',')[0]
    # 특수문자 이후의 텍스트 제거
    text = re.sub(r'[(*&].*$', '', text).strip()
    # 공백 제거
    text = text.replace(' ', '')
    return text

# 데이터프레임에 함수 적용
menu_w_sold_out['first_menu'] = menu_w_sold_out['course'].apply(filter_text)


In [7]:
os.getcwd()

'/Users/al01989093/Desktop/haeyoon_private/cafeteria'

In [8]:
unique_first_menu = pd.DataFrame(menu_w_sold_out[['first_menu']].drop_duplicates()).reset_index(drop=True)
unique_first_menu.to_csv(
         os.getcwd()+'/data/feature_lab/menu_map/v2/unique_first_menu.csv',
         index= False
         )

In [9]:
similarity_df = pd.DataFrame()
for a_menu in tqdm(unique_first_menu.first_menu) :
    similarity_df_by_menu = unique_first_menu.copy()
    similarity_df_by_menu['sim_menu'] = a_menu
    similarity_df_by_menu['similarity'] = similarity_df_by_menu['first_menu'].apply(lambda x: fuzz.partial_ratio(x, a_menu))
    similarity_df = pd.concat([similarity_df,similarity_df_by_menu])

100%|██████████| 663/663 [00:05<00:00, 116.49it/s]


In [41]:
similarity_df.to_csv(
         os.getcwd()+'/data/feature_lab/menu_map/v2/first_menu_similarity.csv',
         index= False
         )

In [30]:
similarity_over_threshold = similarity_df[similarity_df.similarity>=60]

In [68]:
similarity_over_threshold

,first_menu,sim_menu,similarity
0,투움바파스타,투움바파스타,100
387,로제파스타,투움바파스타,60
1,닭곰탕,닭곰탕,100
74,나주곰탕,닭곰탕,67
144,미나리나주곰탕,닭곰탕,67
...,...,...,...
279,버섯콩나물밥,취나물버섯밥,67
438,취나물버섯영양밥,취나물버섯밥,83
497,취나물밥,취나물버섯밥,75
625,취나물,취나물버섯밥,100


In [59]:
# 일단 저녁 부터 진행하자 
menu_w_sold_out['menu_no'] = menu_w_sold_out.groupby(['day', 'meal_time'])['day'].transform('size')
target_df = menu_w_sold_out[(menu_w_sold_out.meal_time =='dinner') & 
                     (menu_w_sold_out.menu_no>1)& # 해당일에 2가지 이상의 메뉴가 있는 경우만! 
                     (menu_w_sold_out.day>='2023-06-01')].reset_index(drop=True)
menu_w_sold_out['day'] = pd.to_datetime(menu_w_sold_out['day'], format='%Y-%m-%d') # 날짜 형식으로 변환


In [61]:
# 6개월 이내 비슷한 메뉴가 몇번 이나 나왔나? (점심/저녁 포함)
target_df['similar_menu'] = 0
# a. 6개월 이내 저녁에서 몇번이나 나왔나? (2개이상 메뉴가 나오는 경우)
target_df['tot_count_dinner'] = 0
# b. a중 솔드아웃이 얼마나?
target_df['soldout_count_dinner'] = 0

# 6개월 이내 석식에서 품절이 몇번이나 있었나? 
for i in tqdm(range(0,2)):#target_df.shape[0]
    
    target_date = target_df.day[i]
    before_6_months = target_date - relativedelta(months=6)

    target_menu = target_df.first_menu[i]
    
    total_l_d = menu_w_sold_out[
                     (menu_w_sold_out.day<target_date) & 
                     (menu_w_sold_out.day>=before_6_months)
                     ]

    total_d = menu_w_sold_out[
                     (menu_w_sold_out.meal_time =='dinner') & 
                     (menu_w_sold_out.menu_no>1)& # 해당일에 2가지 이상의 메뉴가 있는 경우만! 
                     (menu_w_sold_out.day<target_date) & 
                     (menu_w_sold_out.day>=before_6_months)
                     ]
    

100%|██████████| 2/2 [00:00<00:00, 408.32it/s]


In [70]:
target_menu

'불고기핫도그'

In [69]:
similarity_over_threshold[similarity_over_threshold.sim_menu == target_menu]

,first_menu,sim_menu,similarity
4,매실청돈육불고기,불고기핫도그,67
16,뚝배기소불고기,불고기핫도그,67
47,반반불고기,불고기핫도그,60
63,파채버섯소불고기,불고기핫도그,67
70,불고기핫도그,불고기핫도그,100
81,핫도그떡볶이,불고기핫도그,67
118,뚝배기버섯소불고기,불고기핫도그,67
120,매콤콩나물돈육불고기,불고기핫도그,67
170,돈육생강장불고기,불고기핫도그,67
209,돈육고추장불고기,불고기핫도그,67
